In [ ]:
from pathlib import Path
import sys
import tensorflow as tf
from keras import Sequential, layers
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.applications import EfficientNetB0
from bee_tector.data import load_datasets, undersample_dataset

ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))


train_ds, val_ds, test_ds = load_datasets()

train_ds, id_to_class = undersample_dataset()


def ensure_rgb(image, label):
    if image.shape[-1] != 3:
        image = tf.image.grayscale_to_rgb(image)
    return image, label

train_ds = train_ds.map(ensure_rgb)
val_ds = val_ds.map(ensure_rgb)
test_ds = test_ds.map(ensure_rgb)


def initialize_model(shape=(224, 224, 3), num_classes=12):
    base_model = EfficientNetB0(
        include_top=False,
        input_shape=shape,
        weights=None
    )
    base_model.trainable = True

    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(1e-4),
        metrics=['accuracy']
    )
    return model


model = initialize_model(shape=(224,224,3), num_classes=len(id_to_class))


es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


history = model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[es], verbose=1)


val_loss, val_acc = model.evaluate(val_ds)
test_loss, test_acc = model.evaluate(test_ds)

print(f"Validation accuracy: {val_acc:.4f}, Test accuracy: {test_acc:.4f}")


2025-09-01 17:53:10.486087: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-01 17:53:35.026055: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Found 3619 files belonging to 12 classes.


2025-09-01 17:53:46.314436: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Found 781 files belonging to 12 classes.
Found 781 files belonging to 12 classes.
Epoch 1/10


2025-09-01 17:56:34.726556: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 154140672 exceeds 10% of free system memory.
2025-09-01 17:56:35.799216: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 154140672 exceeds 10% of free system memory.
2025-09-01 17:56:36.557728: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 154140672 exceeds 10% of free system memory.
2025-09-01 17:56:36.766029: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 154140672 exceeds 10% of free system memory.
2025-09-01 17:56:37.643251: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 156905472 exceeds 10% of free system memory.
